In [ ]:
#Python Program for DPGWO
#step 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import random
import math
import time
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

#step 2  loading data set
ds_tar1=pd.read_csv('d:/msk/jack data/target_all.csv') #Table 1 Target Datasets of 115 crack Images
ds_fea=pd.read_csv('d:/msk/jack data/all_features.csv') #Table 2 Features Datasets of 115 crack Images

ds_tar = ds_tar1.iloc[:,0] #for length change 1 to depth and 2 to width

#Final DPGWO with life count and population control for demonstration

fea=pd.DataFrame()
fea1=pd.DataFrame()
dpro = 0.05 # propability for disease 
n_sample = 10 ###number of fireflies
m_sample = 20 ###maximum sample number
nof = 15 #no. of features
nitr = 20 #number of iterations
selpr =0.45 #probability for selection for reproduction
fnc1=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21','f22']#feature column names
fnc = fnc1[:nof]
trial = 'T1' ###trail number
ps_tr = []
rs_tr = []
ps_tt = []
rs_tt = []
so = []
feano = pd.DataFrame()

mdl = XGBRegressor()
#population initialozation and calculation of objective function
for i in range(n_sample):
    #print('Sample No.',i)
    fea[i]=pd.Series(random.sample(range(0, 22), nof)) 
feat = fea.T
for i in range(fea.shape[1]):
    for j in range(fea.shape[0]):
        feat.iloc[i,j] = fea.iloc[j,i]
#generation of wolf life
li=[]
rp=[]
for i in range(n_sample):
    li.append(random.randint(0,nitr))
    rp.append(0)

li = pd.Series(li)
rp = pd.Series(rp)
#evaluation of population
for i in range(fea.shape[1]):
    ds_sfea = ds_fea.iloc[:,fea[i]]
    xtr, xtt, ytr, ytt = train_test_split(ds_sfea, ds_tar, test_size = 0.2, random_state = 2)
    mdl.fit(xtr, ytr)
    ytr_p = mdl.predict(xtr)
    ytt_p = mdl.predict(xtt)
    rs_tr.append(r2_score(ytr, ytr_p))
    ps_tr.append(np.sqrt(mean_squared_error(ytr, ytr_p)))
    rs_tt.append(r2_score(ytt, ytt_p))
    ps_tt.append(np.sqrt(mean_squared_error(ytt, ytt_p)))
    
pstr = pd.Series(ps_tr)
pstt = pd.Series(ps_tt)
rstr = pd.Series(rs_tr)
rstt = pd.Series(rs_tt)
per = pd.concat([pstr, pstt, rstr, rstt], axis=1)
per.columns=['pstr','pstt','rstr','rstt']

feat.columns=fnc
so = pd.concat([per, feat, li, rp], axis = 1)

so['Obj']=so.iloc[:,0]*0.25 + so.iloc[:,1]*0.25 + so.iloc[:,2]*0.25 + so.iloc[:,3]*0.25
so[fnc]=so[fnc].astype('int64')
so1 = so
so1[fnc]=so1[fnc].astype('int64')
#selecting the best one
so1.sort_values('Obj', ascending = False)
rslt = so1.iloc[0,:]
rslt = pd.DataFrame(rslt)
print('Population Initialization\n----------------------\n',so)
nso = so;
#initialization of GWO parameters
sf=2 #scaling factor
als = 0
bes = 0
des = 0

s1=[]
s2=[]
s3=[]
d_sampe = n_sample #dynamic sample
iso = so
ps_tr2 = []
rs_tr2 = []
ps_tt2 = []
rs_tt2 = []
for it in range(nitr):
    iso = so
    print('Iteration ',it)
    #removing dead wolf 
    selg=[]
    selr=[]
    selrg=[]
    d_sample = iso.shape[0]
    print('Before dead wolf no. of samples=',iso.shape[0],'\n',iso)
    for i in range(iso.shape[0]):
        if it!=iso.iloc[i,-3]:
            selg.append(i)

    sog = iso.iloc[selg,:]
    #sample size after removing the dead wolf
    
    print('After dead wolf sample no.=',sog.shape[0], '\n', sog)
    #check the population ----- doesn't exceed the maximum population
    if sog.shape[0] > m_sample:
        mpop =[]
        for i in range(sog.shape[0]):
            if random.random() > dpro:
                    mpop.append(i)
        sog = sog.iloc[mpop,:]
    # control of exceeding population ----- over here
    #reproduction process starts----------------
    if it > int(0.25*nitr):
        selrg = []
        print('\nReproduction process')
        for i in range(iso.shape[0]):
            if iso.iloc[i,-2] > int(0.25*nitr): 
                if random.random() < selpr:
                    selrg.append(i)
        selrg = pd.Series(selrg)
        print('No. of child = ',len(selrg), selrg.shape[0])
        #generating new solutions
        if len(selrg)!=0:
            for i in range(selrg.shape[0]):
                print('Child sample No.',i)
                fea1[i]=pd.Series(random.sample(range(0, 22), nof)) 
            feat1 = fea1.T
            
            for i in range(len(selrg)):
                for j in range(fea1.shape[0]):
                    feat1.iloc[i,j] = fea1.iloc[j,i]
                    
            #generation of child wolf life
            li1=[]
            rp1=[]
            for i in range(len(selrg)):
                lif = random.randint(0,nitr-it)
                if lif == it:
                    lif = lif + 1
                li1.append(lif)
                rp1.append(0)

            li1 = it + pd.Series(li1)
            rp1 = pd.Series(rp1)
            #evaluation of child wolf
            so2 = pd.DataFrame()
            per2 = pd.DataFrame()
            ps_tr2 = []
            rs_tr2 = []
            ps_tt2 = []
            rs_tt2 = []
            for i in range(len(selrg)):
                ds_sfea2 = ds_fea.iloc[:,fea1[i]]
                xtr2, xtt2, ytr2, ytt2 = train_test_split(ds_sfea2, ds_tar, test_size = 0.2, random_state = 2)
                mdl.fit(xtr2, ytr2)
                ytr_p2 = mdl.predict(xtr2)
                ytt_p2 = mdl.predict(xtt2)
                rs_tr2.append(r2_score(ytr2, ytr_p2))
                ps_tr2.append(np.sqrt(mean_squared_error(ytr2, ytr_p2)))
                rs_tt2.append(r2_score(ytt2, ytt_p2))
                ps_tt2.append(np.sqrt(mean_squared_error(ytt2, ytt_p2)))
            pstr2 = pd.Series(ps_tr2)
            pstt2 = pd.Series(ps_tt2)
            rstr2 = pd.Series(rs_tr2)
            rstt2 = pd.Series(rs_tt2)
            per2 = pd.concat([pstr2, pstt2, rstr2, rstt2], axis=1)
            per2.columns=['pstr','pstt','rstr','rstt']
            feat1.columns=fnc
            so2 = pd.concat([per2, feat1, li1, rp1], axis = 1)
            so2['Obj']=so2.iloc[:,0]*0.25 + so2.iloc[:,1]*0.25 + so2.iloc[:,2]*0.25 + so2.iloc[:,3]*0.25
            print('No of child=\n',len(selrg),'child details\n',so2.iloc[:len(selrg),:])
            so2[fnc]=so2[fnc].astype('int64')
            #print('No of child=\n',so2.shape[0],'child details\n',so2)
            #combining Selection for repreoduction
            noro = len(selrg)
            so2 = so2.iloc[:len(selrg),:]
            sog = pd.concat([sog, so2], axis = 0)  
            #d_sample=sog.shape[0]
            print('Sample size after child generation so2', sog)
  
    
    for i in range(sog.shape[0]):
        #print('\nsog.iloc',sog.iloc[i,-1],'\nals',als,'soc.iloc all',sog.iloc[i,:])
        if sog.iloc[i,-1] > als:
            als = sog.iloc[i,-1]
            alp = sog.iloc[i,4:-3]
            alo = sog.iloc[i,-1]
            #print('alp=',alp)
        if sog.iloc[i,-1] < als and sog.iloc[i,-1] > bes:
            bes = sog.iloc[i,-1]

            bep = sog.iloc[i,4:-3]
            beo = sog.iloc[i,-1]
        if sog.iloc[i,-1] < als and sog.iloc[i,-1] < bes and sog.iloc[i,-1] > des:
            des = sog.iloc[i,-1]
            dep = sog.iloc[i,4:-3]
            deo = sog.iloc[i,-1]  
    a = sf - it *(sf/nitr) #a decreasing linearly from 2 to 0
    nso = sog
    for i in range(sog.shape[0]):
 
        pos = sog.iloc[i,4:-3]
        for j in range(nof):
            r1=random.random()
            r2 = random.random()
            A1 = 2*a*r1-a
            C1 = 2*r2
            D_al = np.abs(C1*alp[j]- pos[j])
            X1 = alp[j]-A1*D_al
            r3 = random.random()
            r4 = random.random()
            A2=2*a*r3-a
            C2=2*r4
            D_be = np.abs(C2*bep[j]- pos[j])
            X2 = bep[j]-A2*D_be
            r5 = random.random()
            r6 = random.random()
            A3=2*a*r5-a
            C3=2*r6
            D_de = np.abs(C3*dep[j]- pos[j])
            X3 = dep[j]-A3*D_de
            pos[j] = (X1+X2+X3)/3
            if pos[j] > 21: #position greater than number of feature 
                pos[j] = 21
            if pos[j] < 0: #position less than zero
                pos[j]=0
        npo = pd.Series(pos)
        npo.columns = fnc

        npo[fnc]=npo[fnc].astype('int64')
      
        #checking repetition of features if any
        s1=[]
        s2=[]
        s3=[]
        s1=npo
        s2=[*set(s1)]
                
        [s3.append(x) for x in list(range(22)) if x not in s2]
        len1 = nof - len(s2)
        for i1 in range(len1):
            s2.append(s3[i1])
                
        npo2=pd.Series(s2)

        ###feature selection
        ds_sfean = ds_fea.iloc[:,s2]
        xtr1, xtt1, ytr1, ytt1 = train_test_split(ds_sfean, ds_tar, test_size = 0.2, random_state = 2)
        #feature evaluation
        mdl.fit(xtr1, ytr1)
        ytr_p1 = mdl.predict(xtr1)
        ytt_p1 = mdl.predict(xtt1)
        rs_tr1=(r2_score(ytr1, ytr_p1))
        ps_tr1=(np.sqrt(mean_squared_error(ytr1, ytr_p1)))
        rs_tt1=(r2_score(ytt1, ytt_p1))
        ps_tt1=(np.sqrt(mean_squared_error(ytt1, ytt_p1)))
        obj1 = ps_tr1*0.25 + ps_tt1 *0.25 + rs_tr1*0.25 + rs_tt1 *0.25 
        nso.iloc[i,4:-3] = npo2
        nper = pd.Series([ps_tr1,ps_tt1,rs_tr1,rs_tt1])
        nso.iloc[i,0:4] = nper
        nso.iloc[i,-1] = obj1
        #print('new soln updated')
    nso1 = nso
    nso1.sort_values('Obj',ascending = False)
    bst1 = nso1.iloc[0,:]

    rslt[it+1] = bst1
    so = nso
    so.iloc[:,-2] = 1 + so.iloc[:,-2]
    #print('New solutionp:',nso)
####------------convergence plot-------------
rslt1 = rslt.T
rslt1=pd.DataFrame(rslt1)
fna = 'd:/msk/jack data/dpgwo_jack_nof' + str(nof) + '26523.xlsx'
rslt1.to_excel(fna)
print(rslt1.shape[0])
ptr=[rslt1.iloc[0,0]]
ptt=[rslt1.iloc[0,1]]
rtr=[rslt1.iloc[0,2]]
rtt=[rslt1.iloc[0,3]]
sobj=[rslt1.iloc[0,-1]]

for i in range(rslt1.shape[0]):
    if rslt1.iloc[i,0] < ptr[i]:
        ptr.append(rslt1.iloc[i,0])
    else:
        ptr.append(ptr[i])
        
    if rslt1.iloc[i,1] < ptt[i]:
        ptt.append(rslt1.iloc[i,1])
    else:
        ptt.append(ptt[i])
    if rslt1.iloc[i,2] > rtr[i]:
        rtr.append(rslt1.iloc[i,2])
    else:
        rtr.append(rtr[i])
    if rslt1.iloc[i,3] > rtt[i]:
        rtt.append(rslt1.iloc[i,3])
    else:
        rtt.append(rtt[i])
    if rslt1.iloc[i,-1] > sobj[i]:
        sobj.append(rslt1.iloc[i,-1])
    else:
        sobj.append(sobj[i])        

x = range(len(ptr))
f1, ax1 = plt.subplots()
f2, ax2 = plt.subplots()
f3, ax3 = plt.subplots()

ax1.plot(x, ptr,'g-')
ax1.plot(x, ptt,'b-')
ax1.set_xlabel('Iteration Number')
ax1.set_ylabel('RMSE')

ax1.legend(['Training Dataset', 'Testing Dataset'])
ax2.plot(x, np.abs(rtr),'r-')
ax2.plot(x, np.abs(rtt),'m-')
ax2.set_xlabel('Iteration Number')
ax2.set_ylabel('${R{^2}}$')

ax2.legend(['Training Dataset', 'Testing Dataset'])    
ax3.plot(x, sobj, color='g')
ax3.set_xlabel('Iteration Number')
ax3.set_ylabel('Objective Value')